In [1]:
#import modules
import numpy as np
from math import sqrt

#nlp text processing dependencies
import re
import string
import inflector
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


#pyspark dependencies
import pyspark
from pyspark.sql.functions import *
from pyspark.ml.clustering import KMeans
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import Word2Vec
from pyspark import SQLContext
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, FloatType, DoubleType


In [2]:
#define spark context
spark = SparkSession\
        .builder\
        .appName("SentimentAnalysis")\
        .getOrCreate()

sc = spark.sparkContext


In [3]:
sqlContext = SQLContext(sc)

In [ ]:
# Imported data from tweepy related to COVID-19 hasthags to a CSV
# Cleaned/Structured/Preprocessed the data by performing the following steps:
# 1. converted text to lower case to make it case insensitive
# 2. URLS included in the tweets were removed
# 3. User mentions were removed
# 4. Words were tokenized
# 5. Removed words containing only digits
# 6. Stop words were removed
# 7. Punctuations were removed,except # and *, because * is included in swear words; and if # is removed then 
#    hashtag related word also gets lemmatized which doesn't make any sense. Therefore, to retain the integrity 
#    of the sentence # is retained.
# 8. Words are lemmatized
# 
# Building of word2vec from the cleaned and processed dataset, with a vector size of 100. And minimum 
# frequency of 100.
# 
# The vector from word2vec modelare used as features for k-means clustering model. The k-means model is implemented 
# with k=2 (2 clusters) so that we get positive and negative words clustered seperately.
# 
# The distance of the words in the word2vec model from the 2 cluster centers is calculated, and are sorted based on 
# the shortest distance from the cluster. 
# Upon manual inference of the top 100 words for each cluster, we find that one cluster is more positive and the other 
# is more negative.
#
#
# Scope: We see that some of the words despite being positive, occur in the negative cluster and vice versa. With a 
# larger dataset, the results can be improved.
#
# 

In [5]:
# load csv data
raw_dataframe = spark.read.format("csv") \
.option("header", "true") \
.load("/Users/agarwalshruti28/Downloads/Twitter\ Analysis/coronavirus-covid19-tweets/2020-03-1\ Coronavirus\ Tweets.CSV")

#print Schema
raw_dataframe.printSchema()


root
 |-- status_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- reply_to_status_id: string (nullable = true)
 |-- reply_to_user_id: string (nullable = true)
 |-- reply_to_screen_name: string (nullable = true)
 |-- is_quote: string (nullable = true)
 |-- is_retweet: string (nullable = true)
 |-- favourites_count: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- place_full_name: string (nullable = true)
 |-- place_type: string (nullable = true)
 |-- followers_count: string (nullable = true)
 |-- friends_count: string (nullable = true)
 |-- account_lang: string (nullable = true)
 |-- account_created_at: string (nullable = true)
 |-- verified: string (nullable = true)
 |-- lang: string (nullable = true)



In [6]:
#register temp table for sql usage
raw_dataframe.registerTempTable("raw_dataframe")

In [7]:

# filter only english tweets and get status_id and text for pr
english_tweets_raw = spark.sql("""SELECT status_id, text

                     FROM raw_dataframe
                     
                     WHERE lang="en"
                   """ )



In [8]:
# top 20 rows
english_tweets_raw.show()

+-------------------+--------------------+
|          status_id|                text|
+-------------------+--------------------+
|1237891051307896832|Tonight: Wednesda...|
|1237891052159119366|No, but seriously...|
|1237891052779884545|To help those in ...|
|1237891053182738432|Today in the ente...|
|1237891053686075392|This JAMA Insight...|
|1237891054600241152|Watch this newly ...|
|1237891054721880067|Think about alter...|
|1237891054797352964|With 200 colleges...|
|1237891055199981578|Get the latest on...|
|1237891055502020612|Do you trust your...|
|1237891060879118337|Shocking X-ray im...|
|1237891072358903813|This is a stagger...|
|1237891073503944705|Fucking moron #Wo...|
|1237891074539974665|University of Cop...|
|1237891075445903363|5 Pack Face Masks...|
|1237891075970412545|Aye @realDonaldTr...|
|1237891078105284608|Small gadget to f...|
|1237891079057190913|On https://t.co/p...|
|1237891085281546240|As a mom, I’ve ca...|
|1237891087764619266|$ALT -IMO with on...|
+----------

In [9]:
#total number of english tweets
english_tweets_raw.count()

167449

In [10]:
english_tweets_raw.select('text').show(1)

+--------------------+
|                text|
+--------------------+
|Tonight: Wednesda...|
+--------------------+
only showing top 1 row



In [11]:
def clean_tweets_text(text):
    '''
    Clean Tweet text for Processing
    1. Remove URLS
    2. Remove User Mentions
    3. Expand english contraction: isn't -> is not
    3. Remove Punctuations Except Star,Hyphen & Hashtags
    4. Lower Case
    5. Tokenize Words
    6. Remove stop words
    7. Normalizing Text using Lemmatizer
    8. Restructure Sentence
 
    
    Parameters: 
    
    text (str): raw text
  
    Returns: 
    
    clean_text (str): clean_text

    '''
    
    #Lower Case
    clean_text=text.lower()
    
    
    
    #Remove URLS
    clean_text=re.sub(r'http\S+', '', clean_text)
    
    #Remove User Mentions
    clean_text = re.sub(r'@\w+ ?', '', clean_text)
    
    
    
    
    #special case
    clean_text=clean_text.replace("’","'")
    
    
    
    
    #Tokenize Words
    words_list=clean_text.split(" ")
    words_split = [word for word in words_list if ((word.strip()!='')&(word.strip().isdigit()==False))]
    
    
    
    #Remove stop words
    clean_text = (" ").join([word for word in words_split if word not in stopwords.words('english')])

    
    
    
    #Remove Punctuations except *
    remove = string.punctuation
    remove = remove.replace("*", "") # don't remove star
    remove = remove.replace("#", "") # don't remove #
    pattern = r"[{}]".format(remove) # create the pattern
    clean_text=re.sub(pattern, " ", clean_text) 
    
    #split words
    words_list=clean_text.split(" ")
    
    #Lemmatize Words 
    lemmatizer = WordNetLemmatizer()
    processed_word_list=[lemmatizer.lemmatize(word, pos='v').strip() for word in words_list if ((word.strip()!='')&(len(word.strip())>2)&(word.strip().isdigit()==False))]
    
    
    
    return [word for word in processed_word_list if word not in stopwords.words('english')]

In [17]:
#define python functions as pysaprk sql

clean_tweets_text_udf=udf(lambda text: clean_tweets_text(text), ArrayType(StringType()))

In [18]:
english_tweets_processed=english_tweets_raw.withColumn('text_processed',clean_tweets_text_udf('text'))

In [21]:
english_tweets_processed.show(50,truncate=False)

+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|status_id          |text                                                                                                                                                                                                                                                                                                                |text_processed                                                                          

In [22]:
word2Vec = Word2Vec(vectorSize=100, minCount=100, inputCol="text_processed", outputCol="features")
model = word2Vec.fit(english_tweets_processed)

In [23]:
result = model.transform(english_tweets_processed)

In [24]:

numIterations = 3
numberClusters = 2
kmeans = KMeans().setMaxIter(numIterations).setK(numberClusters).setSeed(42)
kmeans_model = kmeans.fit(result)

In [25]:
predictions = kmeans_model.transform(result)

In [26]:
predictions.select("text","text_processed","prediction").show(10,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|text                                                                                                                                                                                                                                                                                                                |text_processed                                                                                                                                    

In [27]:
word2Vec_df=model.getVectors()

In [28]:
word2Vec_df.show(10)

+----------+--------------------+
|      word|              vector|
+----------+--------------------+
|   serious|[-0.3350479602813...|
|    #ausgp|[-0.0515515431761...|
|  terrible|[-0.0426084846258...|
|      rate|[0.42698106169700...|
|inevitable|[0.00954001024365...|
|      snow|[-0.0769913196563...|
|   michael|[0.25942403078079...|
|california|[0.07665679603815...|
|     scare|[-0.2931016087532...|
|  scenario|[0.04100361466407...|
+----------+--------------------+
only showing top 10 rows



In [29]:
word2Vec_df.count()

2300

In [30]:
def get_word_distance_from_cluster_center(cluster_center_array,word2vec_array):
    euclidean_distance = np.linalg.norm(cluster_center_array-word2vec_array)
    return euclidean_distance

In [31]:
word2Vec_df=word2Vec_df.toPandas()
word2Vec_df["vector"]=word2Vec_df["vector"].apply(lambda x: np.array(x))


In [32]:
word2Vec_df["Distance from Cluster Center 0"]=word2Vec_df['vector'].apply(lambda x: get_word_distance_from_cluster_center(kmeans_model.clusterCenters()[0],x))

In [33]:
word2Vec_df["Distance from Cluster Center 1"]=word2Vec_df['vector'].apply(lambda x: get_word_distance_from_cluster_center(kmeans_model.clusterCenters()[1],x))

In [38]:
word2Vec_df[["word","Distance from Cluster Center 0"]].sort_values("Distance from Cluster Center 0")[:50]

word  Distance from Cluster Center 0
962      #arteta                        0.563547
214       #bbcqt                        0.583213
862        ghana                        0.603011
185    #broadway                        0.638844
592       strike                        0.658496
2018        king                        0.693372
1969    #arsenal                        0.701917
362          con                        0.713639
2112        lift                        0.741400
1219       #ohio                        0.757882
1839   celebrate                        0.765276
1895       hello                        0.769511
212   apparently                        0.785654
204         fair                        0.791623
928         ring                        0.791885
1564       texas                        0.795770
92           art                        0.803922
1399    friendly                        0.809916
382      houston                        0.819910
1874        road                        0.821362
2235   otherwise                        0.823698
1940        camp                        0.837628
19        launch                        0.842863
418   incredible                        0.848644
2170         sum                        0.851645
514          btw                        0.852711
187         poll                        0.852965
1816      longer                        0.854487
417   impossible                        0.855104
1273        also                        0.858414
813        sadly                        0.859058
1643     traffic                        0.870293
1476       thats                        0.872380
1552       grant                        0.873619
1612         cop                        0.875541
1932        ride                        0.875696
1452   obviously                        0.879333
2129       along                        0.891645
282      subject                        0.893740
159        irony                        0.896529
1501     curious                        0.897265
5           snow                        0.897739
665    wonderful                        0.897866
670    somewhere                        0.899133
1834         psa                        0.899996
373    professor                        0.900358
2223      beyond                        0.904211
1353   therefore                        0.904407
718      awesome                        0.910206
915       sister                        0.914144

In [37]:
word2Vec_df[["word","Distance from Cluster Center 1"]].sort_values("Distance from Cluster Center 1")[:50]

word  Distance from Cluster Center 1
214        #bbcqt                        0.435903
1895        hello                        0.595552
962       #arteta                        0.658104
1612          cop                        0.676556
592        strike                        0.683018
862         ghana                        0.702851
159         irony                        0.714536
718       awesome                        0.717836
928          ring                        0.719447
1839    celebrate                        0.720573
1932         ride                        0.721769
670     somewhere                        0.724861
417    impossible                        0.725997
92            art                        0.727310
2235    otherwise                        0.728883
1452    obviously                        0.729626
514           btw                        0.734771
1399     friendly                        0.735769
2170          sum                        0.736862
766          tear                        0.738362
470         style                        0.741397
541      normally                        0.754582
2018         king                        0.756126
362           con                        0.761123
1969     #arsenal                        0.765395
2085        legit                        0.767108
212    apparently                        0.773917
1834          psa                        0.775904
1155   priorities                        0.777970
2112         lift                        0.780224
1476        thats                        0.780727
813         sadly                        0.785335
1966         none                        0.790998
665     wonderful                        0.791123
418    incredible                        0.795432
2295    brilliant                        0.797277
1940         camp                        0.802791
182         crack                        0.807527
615          folk                        0.808238
897    capitalism                        0.809235
261       replace                        0.811136
185     #broadway                        0.813278
1273         also                        0.814908
688     nightmare                        0.815241
956        bother                        0.819887
1077   personally                        0.823117
1542  environment                        0.823313
804         awful                        0.824753
1150   incredibly                        0.826116
140        random                        0.829606

In [41]:
predictions.select('prediction').groupby('prediction').count().show()

+----------+------+
|prediction| count|
+----------+------+
|         1|133658|
|         0| 33791|
+----------+------+



In [ ]:
# 1 is the negative cluster
# 0 is the positive cluster